In [10]:
import json
import os
import shutil
import time
from base64 import b64decode
from io import BytesIO
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import PIL
import seaborn as sns
#%config InlineBackend.figure_format = 'svg'  

@register_cell_magic
def capture_png(line, cell):
    get_ipython().run_cell_magic(
        'capture',
        ' --no-stderr --no-stdout result',
        cell
    )


    out_paths = line.strip().split(' ')
    for output in result.outputs:
        display(output)
        
        data = output.data
        if 'image/png' in data:
            path = out_paths.pop(0)
            path = path.split(".png")[0] + str(time.time_ns()) + ".png"
            if not path:
                raise ValueError('Too few paths given!')
            png_bytes = data['image/png']
            if isinstance(png_bytes, str):
                png_bytes = b64decode(png_bytes)
            assert isinstance(png_bytes, bytes)
            bytes_io = BytesIO(png_bytes)
            image = PIL.Image.open(bytes_io)
            image.save(path, 'png')
            
        code =cell

        with open("imgs/structuring_elements.json", "r") as jsonFile:
            data = json.load(jsonFile)

        data[path[5:]] = code

        with open("imgs/structuring_elements.json", "w") as jsonFile:
            json.dump(data, jsonFile, indent=2)




    
def disp(x, sep='.0f', annot=False,  figsize=(5,5)): 
    plt.figure(figsize =figsize)
    plt.axis("off")
    sns.heatmap(x,annot=annot,fmt=sep, cmap="YlGnBu", cbar=False); # Change range with: vmin=0, vmax=1
        
temp_path = Path.cwd() / "temp_images"
temp_path.mkdir(parents=False, exist_ok=True)

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from skimage.morphology import ball, cube, octahedron

def plot_voxels(varray, ax= None, title= None, linewidth=0.8):
    cmap = plt.get_cmap("YlGnBu")

    if ax == None:
        fig = plt.figure(figsize=(16/2, 9/2))
        ax = fig.add_subplot(1, 1,1, projection="3d")
    ax.view_init(20, 200)
    colors = cmap(varray)
    ax.voxels(varray, facecolors=colors, edgecolor="black", linewidth=linewidth)

    ax.axis('off')
    if title != None:
        ax.set_title(title, fontsize=30)





def structuring_element(array, structure, position=None, alignment_to_index="center"):
    """Draw a structuring element.

    Parameters
    ----------
    array: ndarray
        An array where the structuring element will be placed.
    structure: ndarray
        A structuring element.
    position: array_like, optional
        The position of the structuring element in the output image.
        When no position is given, the stucuting element will be drawn in the center of the array.
        If one or more array axis haven an even count of total elements and therefore no integer as center index,
        the center index is rounded down to the next integer.
    alignment_to_index: string, optional
        Can be either "center" or "corner".
    """
    if position == None:
        i, j, k = array.shape
        position = np.array(
            [int(np.ceil(i / 2)), int(np.ceil(j / 2)), int(np.ceil(k / 2))]
        )
    x = position[0]
    y = position[1]
    z = position[2]
    strulen_x = structure.shape[0]
    strulen_y = structure.shape[1]
    strulen_z = structure.shape[2]
    if alignment_to_index == "center":
        halfxA = int(np.floor(strulen_x / 2))
        halfxB = int(np.ceil(strulen_x / 2))
        halfyA = int(np.floor(strulen_y / 2))
        halfyB = int(np.ceil(strulen_y / 2))
        halfzA = int(np.floor(strulen_z / 2))
        halfzB = int(np.ceil(strulen_z / 2))
        array[
            x - halfxA : x + halfxB, y - halfyA : y + halfyB, z - halfzA : z + halfzB
        ] = structure
    if alignment_to_index == "corner":
        array[x : x + strulen_x, y : y + strulen_y, z : z + strulen_z] = structure

    return array

# Structuing elements

In [ ]:
%%capture_png imgs/example.png
arrray = scipy.ndimage.generate_binary_structure(3, 1)
plot_voxels(arrray)

In [ ]:
%%capture_png imgs/example.png
arrray = scipy.ndimage.generate_binary_structure(3, 2)
plot_voxels(arrray)

In [ ]:
%%capture_png imgs/example.png
arrray = scipy.ndimage.generate_binary_structure(3, 3)
plot_voxels(arrray)

In [ ]:
%%capture_png imgs/example.png
arrray = cube(5)
plot_voxels(arrray)

In [ ]:
%%capture_png imgs/example.png
arrray = octahedron(3)
plot_voxels(arrray)

In [ ]:
%%capture_png imgs/example.png
arrray = ball(3)
plot_voxels(arrray)

In [ ]:
%%capture_png imgs/example.png
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

grids = 2
boxs = 5

voxelarray = np.zeros((boxs * grids, boxs * grids, boxs * grids))

i = 1
for xi in range(0, 2):
    for yi in range(0, 2):
        for zi in range(0, 2):
            voxelarray[
                xi * boxs: xi * boxs + boxs,
                yi * boxs: yi * boxs + boxs,
                zi * boxs: zi * boxs + boxs,
            ] = i
            i += 1

voxelarray = np.uint8(voxelarray*255/i)

plot_voxels(voxelarray)

In [ ]:
from skimage import data
voxelarray = data.binary_blobs(length=110, volume_fraction=0.1, n_dim=3, seed=9)
voxelarray= voxelarray[90:,90:,90:]
#plt.imshow(voxelarray[:,:,3])
plot_voxels(voxelarray, linewidth=0.1)